In [ ]:
import pandas as pd
from functools import reduce
import pickle

In [ ]:
power_agency = "Sap et al. -Connotation Frames of Agency and Power.csv"

power_agency = pd.read_csv(power_agency)
power_agency[["agency_1","agency_classifier"]]= power_agency["agency"].str.split("_",expand=True)
power_agency[["power_1","power_classifier"]]= power_agency["power"].str.split("_",expand=True)
power_agency = power_agency.drop(power_agency[["agency","power", "agency_1", "power_1"]], axis= 1)
power_agency=power_agency.drop(columns='agency_classifier')
power_agency = power_agency[power_agency['power_classifier'].isin(['agent','equal']) != True]
power_agency = power_agency.dropna()
power_agency['source']='power'
power_agency.set_axis(['feature', 'weight', 'source'], axis='columns', inplace=True)



In [ ]:
communality = "Lawson et al. - Communality.csv"
communality = pd.read_csv(communality)
communality = communality.drop(columns=['Valence Mean', 'Valence SD','Communality SD'])
communality['source']='communality'
communality.set_axis(['feature', 'weight', 'source'], axis='columns', inplace=True)



agency = "Lawson et al. - Agency Words.csv"
agency = pd.read_csv(agency)
agency = agency.drop(agency.columns[[2,3,4]], axis=1)
agency = agency.rename(columns={'Word':'feature','Agency Mean':'weight'})
agency['source']='agency'



In [ ]:
apperance = "Garg_apperance_words.csv"

apperance = pd.read_csv(apperance)

violence = "Violence_Grievence_Dicitonary.csv"
violence = pd.read_csv(violence)
violence = violence.rename(columns={'weights':'weight'})


In [ ]:
PAC_words = [power_agency, communality, agency, violence, apperance]
df_attempt = reduce(lambda  left,right: pd.merge(left,right,on=['feature'],how='outer'), PAC_words)

In [ ]:
PAC_words = [power_agency, communality, agency, violence, apperance]
result = pd.concat(PAC_words)
result = result.drop(columns='weight')
result = result.drop(columns='weights')

In [ ]:
result.to_csv("full_word_list")

In [8]:
from nltk import word_tokenize, pos_tag, pos_tag_sents
import pandas as pd
df = result
texts = df['feature'].tolist()
tagged_texts = pos_tag_sents(map(word_tokenize, texts))
df['POS'] = tagged_texts
df

,feature,source,POS
8,accompanies,power,"[(accompanies, NNS)]"
15,achieves,power,"[(achieves, NNS)]"
28,admires,power,"[(admires, NNS)]"
29,admits,power,"[(admits, NNS)]"
31,adores,power,"[(adores, NNS)]"
...,...,...,...
20,weak,Apperance,"[(weak, JJ)]"
21,thin,Apperance,"[(thin, NN)]"
22,pretty,Apperance,"[(pretty, RB)]"
23,beautiful,Apperance,"[(beautiful, NN)]"


In [10]:
df3 = df.POS.apply(pd.Series)
df3.columns = ['1', '2','3','4']
df3['a'] = df3['1'].str[1]
df3['b'] = df3['1'].str[0]

In [11]:
df3

,1,2,3,4,a,b
8,"(accompanies, NNS)",NaN,NaN,NaN,NNS,accompanies
15,"(achieves, NNS)",NaN,NaN,NaN,NNS,achieves
28,"(admires, NNS)",NaN,NaN,NaN,NNS,admires
29,"(admits, NNS)",NaN,NaN,NaN,NNS,admits
31,"(adores, NNS)",NaN,NaN,NaN,NNS,adores
...,...,...,...,...,...,...
20,"(weak, JJ)",NaN,NaN,NaN,JJ,weak
21,"(thin, NN)",NaN,NaN,NaN,NN,thin
22,"(pretty, RB)",NaN,NaN,NaN,RB,pretty
23,"(beautiful, NN)",NaN,NaN,NaN,NN,beautiful


In [12]:
new_col = pd.merge(df, df3, left_on = df3['b'], right_on= df['feature'])
new_col = new_col.rename(columns = {'a': 'pos_tag'})
new_col = new_col.drop(columns=['POS','1','2','3','4','b','key_0'])
left = new_col['pos_tag'].str[:1]
left = pd.DataFrame(left)
left = left.rename(columns={'pos_tag':'pos_tag_short'})

In [13]:
new_col

,feature,source,pos_tag
0,accompanies,power,NNS
1,achieves,power,NNS
2,admires,power,NNS
3,admits,power,NNS
4,adores,power,NNS
...,...,...,...
829,fat,Apperance,NN
830,weak,Apperance,JJ
831,thin,Apperance,NN
832,pretty,Apperance,RB


In [14]:
pos_tags = new_col.join(left, on=new_col.index)

In [15]:
new_file_name ="pos_tag_word_list"

with open(new_file_name, 'wb') as handle:
    pickle.dump(pos_tags, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
pos_tags['pos_tag_short'].value_counts()

N    668
J    118
V     38
R     10
Name: pos_tag_short, dtype: int64

Combine LogReg High results

In [23]:
df_10

,feature,2010_c,coef_type,year
0,arrest,-1.826027,lowest,2010
1,soldier,-1.622398,lowest,2010
2,shoot,-1.620356,lowest,2010
3,girlfriend,-1.327853,lowest,2010
4,football,-1.268172,lowest,2010
...,...,...,...,...
995,embassy,-0.215427,lowest,2010
996,evade,-0.215387,lowest,2010
997,dinghy,-0.214939,lowest,2010
998,bird,-0.214428,lowest,2010


In [24]:
#2010
df_10 = pd.read_pickle("RESULTS10_coef_high.pickle")
df_10.rename(columns = {'coef':'2010_c'}, inplace = True)
df_10 = df_10.drop(columns=['coef_type','year'])
#2011
df_11 = pd.read_pickle("RESULTS11_coef_high.pickle")
df_11.rename(columns = {'coef':'2011_c'}, inplace = True)
df_11 = df_11.drop(columns=['coef_type','year'])
#2012
df_12 = pd.read_pickle("RESULTS12_coef_high.pickle")
df_12.rename(columns = {'coef':'2012_c'}, inplace = True)
df_12 = df_12.drop(columns=['coef_type','year'])
#2013
df_13 = pd.read_pickle("RESULTS13_coef_low.pickle")
df_13.rename(columns = {'coef':'2013_c'}, inplace = True)
df_13 = df_13.drop(columns=['coef_type','year'])
#2014
df_14 = pd.read_pickle("RESULTS14_coef_high.pickle")
df_14.rename(columns = {'coef':'2014_c'}, inplace = True)
df_14 = df_14.drop(columns=['coef_type','year'])
#2015
df_15 = pd.read_pickle("RESULTS15_coef_high.pickle")
df_15.rename(columns = {'coef':'2015_c'}, inplace = True)
df_15 = df_15.drop(columns=['coef_type','year'])
#2016
df_16 = pd.read_pickle("RESULTS16_coef_high.pickle")
df_16.rename(columns = {'coef':'2016_c'}, inplace = True)
df_16 = df_16.drop(columns=['coef_type','year'])
#2017
df_17 = pd.read_pickle("RESULTS17_coef_high.pickle")
df_17.rename(columns = {'coef':'2017_c'}, inplace = True)
df_17 = df_17.drop(columns=['coef_type','year'])
#2018
df_18 = pd.read_pickle("RESULTS18_coef_high.pickle")
df_18.rename(columns = {'coef':'2018_c'}, inplace = True)
df_18 = df_18.drop(columns=['coef_type','year'])
#2019
df_19 = pd.read_pickle("RESULTS19_coef_high.pickle")
df_19.rename(columns = {'coef':'2019_c'}, inplace = True)
df_19 = df_19.drop(columns=['coef_type','year'])
#2020
df_20 = pd.read_pickle("RESULTS20_coef_high.pickle")
df_20.rename(columns = {'coef':'2020_c'}, inplace = True)
df_20 = df_20.drop(columns=['coef_type','year'])
#2021
df_21 = pd.read_pickle("RESULTS21_coef_high.pickle")
df_21.rename(columns = {'coef':'2021_c'}, inplace = True)
df_21 = df_21.drop(columns=['coef_type','year'])
#2022
df_22 = pd.read_pickle("RESULTS22_coef_high.pickle")
df_22.rename(columns = {'coef':'2022_c'}, inplace = True)
df_22 = df_22.drop(columns=['coef_type','year'])

In [27]:
df_list = [df_10, df_11, df_12, df_13, df_15, df_16, df_17, df_18, df_19, df_20, df_21, df_22]
#df = pd.merge(df_list, right on='feature', how='outer')

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['feature'],how='outer'), df_list)
df_high = pd.merge(df_merged, new_col, on='feature', how='outer')

In [28]:
#2010
df_10 = pd.read_pickle("RESULTS10_coef_low.pickle")
df_10.rename(columns = {'coef':'2010_c'}, inplace = True)
df_10 = df_10.drop(columns=['coef_type','year'])
#2011
df_11 = pd.read_pickle("RESULTS11_coef_low.pickle")
df_11.rename(columns = {'coef':'2011_c'}, inplace = True)
df_11 = df_11.drop(columns=['coef_type','year'])
#2012
df_12 = pd.read_pickle("RESULTS12_coef_low.pickle")
df_12.rename(columns = {'coef':'2012_c'}, inplace = True)
df_12 = df_12.drop(columns=['coef_type','year'])
#2013
df_13 = pd.read_pickle("RESULTS13_coef_low.pickle")
df_13.rename(columns = {'coef':'2013_c'}, inplace = True)
df_13 = df_13.drop(columns=['coef_type','year'])
#2014
df_14 = pd.read_pickle("RESULTS14_coef_low.pickle")
df_14.rename(columns = {'coef':'2014_c'}, inplace = True)
df_14 = df_14.drop(columns=['coef_type','year'])
#2015
df_15 = pd.read_pickle("RESULTS15_coef_low.pickle")
df_15.rename(columns = {'coef':'2015_c'}, inplace = True)
df_15 = df_15.drop(columns=['coef_type','year'])
#2016
df_16 = pd.read_pickle("RESULTS16_coef_low.pickle")
df_16.rename(columns = {'coef':'2016_c'}, inplace = True)
df_16 = df_16.drop(columns=['coef_type','year'])
#2017
df_17 = pd.read_pickle("RESULTS17_coef_low.pickle")
df_17.rename(columns = {'coef':'2017_c'}, inplace = True)
df_17 = df_17.drop(columns=['coef_type','year'])
#2018
df_18 = pd.read_pickle("RESULTS18_coef_low.pickle")
df_18.rename(columns = {'coef':'2018_c'}, inplace = True)
df_18 = df_18.drop(columns=['coef_type','year'])
#2019
df_19 = pd.read_pickle("RESULTS19_coef_low.pickle")
df_19.rename(columns = {'coef':'2019_c'}, inplace = True)
df_19 = df_19.drop(columns=['coef_type','year'])
#2020
df_20 = pd.read_pickle("RESULTS20_coef_low.pickle")
df_20.rename(columns = {'coef':'2020_c'}, inplace = True)
df_20 = df_20.drop(columns=['coef_type','year'])
#2021
df_21 = pd.read_pickle("RESULTS21_coef_low.pickle")
df_21.rename(columns = {'coef':'2021_c'}, inplace = True)
df_21 = df_21.drop(columns=['coef_type','year'])
#2022
df_22 = pd.read_pickle("RESULTS22_coef_low.pickle")
df_22.rename(columns = {'coef':'2022_c'}, inplace = True)
df_22 = df_22.drop(columns=['coef_type','year'])

In [29]:
df_list = [df_10, df_11, df_12, df_13, df_15, df_16, df_17, df_18, df_19, df_20, df_21, df_22]
#df = pd.merge(df_list, right on='feature', how='outer')

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['feature'],how='outer'), df_list)
df_low = pd.merge(df_merged, new_col, on='feature', how='outer')

In [33]:
df_high.groupby(by=['source']).mean()

,2010_c,2011_c,2012_c,2013_c,2015_c,2016_c,2017_c,2018_c,2019_c,2020_c,2021_c,2022_c
source,,,,,,,,,,,,
Apperance,0.449046,0.856132,1.175642,-0.356762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.191391
Violence,0.425323,0.520718,0.511634,-0.662280,4.167349,7.056416,4.785320,4.887956,NaN,5.051902,3.564016,1.489215
agency,0.311922,0.414127,0.347780,-0.556460,3.237169,7.710151,NaN,NaN,NaN,NaN,3.808024,1.302509
communality,0.496580,0.515557,0.517508,-0.387169,NaN,NaN,NaN,5.295554,NaN,NaN,NaN,1.708131
power,NaN,NaN,NaN,NaN,3.194963,10.091178,4.647688,4.531505,NaN,4.140059,3.840743,NaN


In [32]:
df_low.groupby(by=['source']).mean()

,2010_c,2011_c,2012_c,2013_c,2015_c,2016_c,2017_c,2018_c,2019_c,2020_c,2021_c,2022_c
source,,,,,,,,,,,,
Apperance,NaN,NaN,-0.471591,-0.356762,NaN,NaN,NaN,NaN,NaN,NaN,-4.198273,-1.102006
Violence,-0.555498,-0.676170,-0.647622,-0.662280,-3.261100,-7.777439,-4.887361,-4.441690,-4.694225,-4.887254,-4.255413,-1.515562
agency,-0.540282,-0.685230,-0.645241,-0.556460,-3.425953,NaN,-4.503772,-5.280673,NaN,-4.182370,NaN,-1.242424
communality,-0.297445,-0.403663,-0.383594,-0.387169,NaN,-8.369745,-4.097979,-4.323050,NaN,-4.282894,NaN,-1.152545
power,NaN,NaN,NaN,NaN,NaN,-6.936589,-4.161319,NaN,-4.004334,NaN,-3.867713,-1.376326
